# Top 10 selector

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
df = pd.read_csv('../data/10%sample/2020-Apr.csv_10%.csv')

In [3]:
df.head()

,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session
0,2020-04-01 00:00:03 UTC,view,100068493,2232732093077520756,construction.tools.light,samsung,319.41,635165435,861f2378-076f-4ddd-85e3-9844923d03a9
1,2020-04-01 00:00:09 UTC,view,9200640,2232732104343421549,apparel.scarf,defender,20.19,533896443,6a220235-f4d6-4987-a51e-8f315b3027fc
2,2020-04-01 00:00:14 UTC,view,5100375,2232732103101907535,electronics.clocks,xiaomi,84.94,635164513,c40d1b96-90aa-4cee-b9aa-9475d9c4f17f
3,2020-04-01 00:00:19 UTC,view,13200853,2232732061804790604,furniture.bedroom.bed,NaN,354.96,635166340,f94c2c0d-abee-4a9b-ad1b-47b85b57c036
4,2020-04-01 00:00:31 UTC,view,2702580,2232732091718566220,appliances.kitchen.refrigerators,midea,710.44,514203043,5adc4e92-9475-407a-ac1b-8c5529f3f23e


In [22]:
# df.query("brand == 'apple' and category_code == 'sport bicycle'")  #check the dataframe output
df.query("product_id == 1002544")

,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session
374,2020-04-01 00:27:00 UTC,view,1002544,2232732093077520756,construction.tools.light,apple,411.27,577108118,d7f5ff23-0df0-4d7e-8918-3afe91efc5ab
1013,2020-04-01 01:08:19 UTC,view,1002544,2232732093077520756,construction.tools.light,apple,411.27,633627289,79a1ec45-5f4f-4ace-a886-d3937f0e9895
1145,2020-04-01 01:14:10 UTC,view,1002544,2232732093077520756,construction.tools.light,apple,411.27,535998339,400b1ed4-4331-4b2c-a6c7-f9ee28494676
1243,2020-04-01 01:18:49 UTC,view,1002544,2232732093077520756,construction.tools.light,apple,411.27,520642995,1ffd603b-7986-4bb8-a9c9-12efed4cafeb
1332,2020-04-01 01:23:09 UTC,view,1002544,2232732093077520756,construction.tools.light,apple,411.27,541028902,43032634-5d09-4e95-b532-32e9105b6d12
...,...,...,...,...,...,...,...,...,...
6663365,2020-04-30 23:46:35 UTC,view,1002544,2232732093077520756,construction.tools.light,apple,425.47,513154022,be0ac38d-c9cc-4a84-9fba-7957d5da73e5
6663384,2020-04-30 23:46:51 UTC,view,1002544,2232732093077520756,construction.tools.light,apple,425.47,623499284,3a12baba-ed67-46b0-992f-77d1e368f969
6663484,2020-04-30 23:48:27 UTC,view,1002544,2232732093077520756,construction.tools.light,apple,425.47,641939270,9ada57ee-5f3b-4252-9f68-9414cfdfd0be
6663979,2020-04-30 23:56:56 UTC,view,1002544,2232732093077520756,construction.tools.light,apple,425.47,642837704,c5eae648-b7b0-4cc6-a3a9-8e21a14ab2be


In [5]:
def preprocessing_feat(X):    
    X_preprocessed = X.dropna(subset = ['category_code', 'brand']) #tbd!!
    X_preprocessed = X_preprocessed.drop_duplicates()
    X_preprocessed['category_code'] = X_preprocessed['category_code'].str.replace('.',' ')
    return X_preprocessed

def pricing_criterion(X):
    pricing_guide = X.groupby('category_code')['price'].describe()[["25%", "75%"]].reset_index()
    X_merged = X.merge(pricing_guide, on="category_code", how="right")
    X_merged["price_category"] = X_merged.apply(lambda row: make_column(row), axis=1)
    return X_merged

def make_column(row):
    if row["price"] < row["25%"]:
        return "low"
    elif row["price"] < row["75%"]:
        return "medium"
    else:
        return "high"

def metadata(X):
    X['metadata'] = X[['category_code', 'brand', 'price_category']].apply(lambda x: ' '.join(x), axis = 1)
    return X

def rating(X):
    dct = {'view': 1, 'cart': 3, 'purchase': 5}
    X['rating'] = X['event_type'].map(dct)
    return X

In [6]:
X_prep = preprocessing_feat(df)

/var/folders/6f/vn_j9pqn1qd8rp92y6x0h0hh0000gn/T/ipykernel_22674/3456390047.py:4: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  X_preprocessed['category_code'] = X_preprocessed['category_code'].str.replace('.',' ')


In [7]:
# X_prep.query("brand == 'apple' and category_code == 'sport bicycle'")  #check the dataframe output
X_prep.query("product_id == 4804718")

,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session
2052,2020-04-01 01:49:27 UTC,view,4804718,2232732079706079299,sport bicycle,apple,326.88,622406464,a81aacdb-03aa-490f-89b4-506b7b308f92
2556,2020-04-01 02:04:22 UTC,view,4804718,2232732079706079299,sport bicycle,apple,326.88,635185706,9e535a12-3330-4a20-b4dc-9121d1cb1e0f
2612,2020-04-01 02:06:20 UTC,view,4804718,2232732079706079299,sport bicycle,apple,326.88,583813563,09831f79-5c22-4c15-a5cd-a1695607d5ed
2775,2020-04-01 02:10:52 UTC,view,4804718,2232732079706079299,sport bicycle,apple,326.88,568870622,b12ef4af-6ec4-4727-9e5c-8123a23165fa
2870,2020-04-01 02:13:42 UTC,view,4804718,2232732079706079299,sport bicycle,apple,326.88,592302160,8f8fb2b9-ceea-4c30-9aee-5022ce7747e0
...,...,...,...,...,...,...,...,...,...
6661119,2020-04-30 23:12:58 UTC,view,4804718,2232732079706079299,sport bicycle,apple,305.30,595556052,14fd07ee-6168-44e1-985d-db189a5bae4a
6661718,2020-04-30 23:21:06 UTC,view,4804718,2232732079706079299,sport bicycle,apple,305.30,540017278,4ffa1d64-7bf7-4ae3-9ad2-9ad88cb89143
6662052,2020-04-30 23:26:06 UTC,view,4804718,2232732079706079299,sport bicycle,apple,305.30,621833375,98f96d4e-ee0d-4bea-b48d-a08e76add128
6663088,2020-04-30 23:42:18 UTC,view,4804718,2232732079706079299,sport bicycle,apple,305.30,649421047,b8bc3141-2f4a-481c-b07a-f80cedf7dfab


In [8]:
X_pric = pricing_criterion(X_prep)

In [9]:
X_rating = rating(X_pric)

In [10]:
def metadata(X):
    X['metadata'] = X[['category_code', 'brand', 'price_category']].apply(lambda x: ' '.join(x), axis = 1)
    return X

In [11]:
X_meta = metadata(X_pric)

In [12]:
X_meta.query("brand == 'apple' and category_code == 'sport bicycle'")
# df_m.query("user_session == 'bde905b5-20f1-40ff-8ef6-a60d875e31e0'")

,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session,25%,75%,price_category,rating,metadata
4922064,2020-04-01 00:11:02 UTC,view,4804056,2232732079706079299,sport bicycle,apple,170.50,616593255,f774c12a-5cad-4fb2-b26b-a42d780ec66d,24.43,102.94,high,1,sport bicycle apple high
4922075,2020-04-01 00:35:33 UTC,purchase,4804056,2232732079706079299,sport bicycle,apple,170.50,554985808,dd81a7a6-cde1-4796-80b9-654024a87518,24.43,102.94,high,5,sport bicycle apple high
4922079,2020-04-01 00:51:11 UTC,purchase,4804055,2232732079706079299,sport bicycle,apple,213.62,597879959,f32f9eef-003b-4b09-b1e0-54e682143133,24.43,102.94,high,5,sport bicycle apple high
4922087,2020-04-01 01:09:30 UTC,view,4804055,2232732079706079299,sport bicycle,apple,213.62,555496883,01ee7262-26c8-4e96-86a8-6be9756f24b4,24.43,102.94,high,1,sport bicycle apple high
4922089,2020-04-01 01:10:42 UTC,view,4804055,2232732079706079299,sport bicycle,apple,213.62,555496883,01ee7262-26c8-4e96-86a8-6be9756f24b4,24.43,102.94,high,1,sport bicycle apple high
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5123897,2020-04-30 23:49:29 UTC,view,4804718,2232732079706079299,sport bicycle,apple,305.30,514432133,2580cfa8-42da-4d57-83f5-7212b559eed9,24.43,102.94,high,1,sport bicycle apple high
5123908,2020-04-30 23:53:30 UTC,view,4804056,2232732079706079299,sport bicycle,apple,162.81,631116537,da084dcc-ce4c-44a6-b3c1-59aad8be8cef,24.43,102.94,high,1,sport bicycle apple high
5123910,2020-04-30 23:53:48 UTC,view,4804055,2232732079706079299,sport bicycle,apple,209.75,631116537,da084dcc-ce4c-44a6-b3c1-59aad8be8cef,24.43,102.94,high,1,sport bicycle apple high
5123923,2020-04-30 23:56:30 UTC,view,4804056,2232732079706079299,sport bicycle,apple,162.81,649400584,1bd2441b-7ad3-443e-bf7d-c9d7296eaa67,24.43,102.94,high,1,sport bicycle apple high


In [13]:
def ranking(X, n=5):
    X = pd.DataFrame(X_rating.groupby(by='product_id').rating.sum()).sort_values('rating', ascending=False).reset_index().index[0:n]
    return X
    

In [14]:
# def top_n(product_id, X_rating, n=10):
    
#     """select top n products by ranking'"""
    
#     rating_idx = pd.DataFrame(X_rating.groupby(by='product_id').rating.sum()).sort_values('rating', ascending=False).reset_index().index[0:n]
#     return rating_idx

#     counter = 0
    
#     for i in rating_idx:
#         meta_text = X_rating[X_rating['product_id'] == i][['metadata']].iloc[0,:][0]
        
#         if counter == 0:
#             print(f"Top {n} recommendations for product_id {product_id}:")
#             print(f"{i} - {meta_text} \n")
# #             print("-----------------------")

#         else:
#             print(f"Rec {counter}) {i} - {meta_text}")
        
#         counter += 1

In [15]:
def top_n_overall(product_id, X_rating, n=5):
    
    """select top n products by ranking'"""
    
    rating_idx = pd.DataFrame(X_rating.groupby(by='product_id').rating.sum()).sort_values('rating', ascending=False).index[0:n]
    #     rec_df.sort_values(ranking, ascending=False).index[0:n]
    counter = 0
    
    for i in rating_idx:
        meta_text = X_meta[X_meta['product_id'] == i][['metadata']].iloc[0,:][0]
        
        if counter == 0:
            print(f"Top {n} recommendations for product_id {product_id}:")
            print(f"{i} - {meta_text} \n")
#             print("-----------------------")

        else:
            print(f"Rec {counter}) {i} - {meta_text}")
        
        counter += 1

In [16]:
top_n_overall(49800017, X_rating)

Top 5 recommendations for product_id 49800017:
1005100 - construction tools light samsung low 

Rec 1) 1005212 - construction tools light samsung medium
Rec 2) 100068488 - construction tools light samsung medium
Rec 3) 100068493 - construction tools light samsung medium
Rec 4) 1005160 - construction tools light xiaomi medium


In [17]:
def top_n_brands(X_rating, n=20):
    
    """select top n products by ranking'"""
    
    rating_idx =     pd.DataFrame(X_rating.groupby(['brand']).rating.sum()).sort_values('rating', ascending=False).index[0:n]

    #     rec_df.sort_values(ranking, ascending=False).index[0:n]
    return list(rating_idx)

In [18]:
top_n_brands(X_rating)

['samsung',
 'apple',
 'xiaomi',
 'huawei',
 'lg',
 'oppo',
 'acer',
 'lenovo',
 'defacto',
 'sony',
 'asus',
 'hp',
 'bosch',
 'redmond',
 'artel',
 'arg',
 'polaris',
 'philips',
 'adidas',
 'lucente']

In [19]:
def top_n_by_brand(X_rating, brand, n=10):
    
    """select top n products by brand'"""
    
    rating_idx = pd.DataFrame(X_rating.groupby(['product_id','brand']).rating.sum()).sort_values('rating', ascending=False).reset_index('brand')
    rating_idx = rating_idx[rating_idx['brand']==brand].index[0:n]
#     return rating_idx

    counter = 0

    for i in rating_idx:
            meta_text = X_meta[X_meta['product_id'] == i][['metadata']].iloc[0,:][0]

            if counter == 0:
                print(f"Top {n} recommendations for product_id {brand}:")
                print(f"{i} - {meta_text} \n")
    #             print("-----------------------")

            else:
                print(f"Rec {counter}) {i} - {meta_text}")

            counter += 1

In [20]:
top_n_by_brand(X_rating, brand='apple')

Top 10 recommendations for product_id apple:
1002544 - construction tools light apple medium 

Rec 1) 1005115 - construction tools light apple high
Rec 2) 4804056 - sport bicycle apple high
Rec 3) 1004249 - construction tools light apple high
Rec 4) 1005105 - construction tools light apple high
Rec 5) 1004226 - construction tools light apple high
Rec 6) 1005116 - construction tools light apple high
Rec 7) 4804055 - sport bicycle apple high
Rec 8) 1005118 - construction tools light apple high
Rec 9) 1003306 - construction tools light apple high
